In [1]:
import torch
import time
import numpy as np

import torchvision
import torchvision.transforms as transforms

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
shape = (3,2)
x = torch.rand(shape, dtype=float)
print(x)

tensor([[0.3868, 0.7403],
        [0.6417, 0.5854],
        [0.1191, 0.1833]], dtype=torch.float64)


In [ ]:
y = torch.ones_like(x)
print(y, y.size())

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], dtype=torch.float64) torch.Size([3, 2])


In [ ]:
out = x + y
print(out)
y.add_(x)
print(y)

tensor([[1.3868, 1.7403],
        [1.6417, 1.5854],
        [1.1191, 1.1833]], dtype=torch.float64)
tensor([[1.3868, 1.7403],
        [1.6417, 1.5854],
        [1.1191, 1.1833]], dtype=torch.float64)


In [ ]:
x = np.random.rand(3,2)
print(x)

tensor_x = torch.from_numpy(x)
print(tensor_x)

numpy_x = tensor_x.numpy()
print(numpy_x)

[[0.35990055 0.82630721]
 [0.25775084 0.98181074]
 [0.46863991 0.85011467]]
tensor([[0.3599, 0.8263],
        [0.2578, 0.9818],
        [0.4686, 0.8501]], dtype=torch.float64)
[[0.35990055 0.82630721]
 [0.25775084 0.98181074]
 [0.46863991 0.85011467]]


In [ ]:
x = torch.rand(3,2, requires_grad=True)
print(x)

y = x*10 + 0.1
print(y)

out = torch.max(y)
print(out)

out.backward()
print(x.grad)

tensor([[0.9448, 0.1139],
        [0.1096, 0.7247],
        [0.6129, 0.0045]], requires_grad=True)
tensor([[9.5477, 1.2387],
        [1.1963, 7.3470],
        [6.2293, 0.1450]], grad_fn=<AddBackward0>)
tensor(9.5477, grad_fn=<MaxBackward1>)
tensor([[10.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]])


In [2]:
#--------------------------------------------------
#       Define Network Architecture
#--------------------------------------------------
class TNet(nn.Module):
    def __init__(self):
      super(TNet,self).__init__()  

      self.conv1 = nn.Conv2d(3, 64, 3) 
      self.layer1 = torch.nn.Sequential(
        nn.ReLU(),
        nn.MaxPool2d(4, stride=4), 
      )
      self.conv2 = nn.Conv2d(64, 128, 3)
      self.layer2 = torch.nn.Sequential(

        nn.Sigmoid(),
        nn.MaxPool2d(2, stride=2), 
      )
      
      self.classifier = nn.Sequential(
         nn.Linear(512, 16), 
      )
      self.dropout = nn.Dropout(0.5)
      
    def forward(self, x):
      x = self.conv1(x)
      x = self.layer1(x)
      x = self.conv2(x)
      x = self.layer2(x)  
      x = torch.flatten(x, 1)
      x = self.dropout(x)
      x = self.classifier(x)
      return x  

model = TNet()
#model = model.cuda()

print (len(list(model.parameters())))

6


In [3]:
input = torch.randn(1, 3, 32, 32)
out = model(input)

print(out)


tensor([[-0.2607, -0.7261,  0.5427,  0.1701, -0.4174, -0.3128, -0.6538, -0.7719,
          0.7229, -0.1859,  0.6547,  0.1600,  0.0373, -0.2867, -0.0388,  0.2156]],
       grad_fn=<AddmmBackward>)


In [4]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.01)

target = torch.randn(16)  
target = target.view(1, -1)  
criterion = nn.MSELoss()


out = model(input)

loss = criterion(out, target)
print(loss)



model.zero_grad()
print('conv1.bias.grad before backward')

print(model.conv1.bias.grad)

loss.backward()

optimizer.step()

print('layer 2.bias.grad after backward')
print(model.conv2.bias.grad)


tensor(1.2815, grad_fn=<MseLossBackward>)
conv1.bias.grad before backward
None
layer 2.bias.grad after backward
tensor([ 0.0034, -0.0040,  0.0000, -0.0003,  0.0239,  0.0015,  0.0097, -0.0021,
        -0.0091, -0.0139,  0.0000,  0.0015,  0.0097, -0.0138,  0.0047, -0.0092,
         0.0083,  0.0153, -0.0112,  0.0035,  0.0187, -0.0122,  0.0075,  0.0153,
        -0.0075, -0.0056,  0.0035, -0.0042,  0.0155,  0.0105,  0.0103,  0.0105,
        -0.0029,  0.0069,  0.0160,  0.0007,  0.0102,  0.0021,  0.0276,  0.0025,
         0.0120, -0.0113,  0.0084, -0.0005,  0.0066,  0.0048, -0.0066,  0.0006,
         0.0269,  0.0125, -0.0098, -0.0013,  0.0081, -0.0045,  0.0148, -0.0059,
        -0.0044,  0.0091,  0.0102, -0.0115,  0.0015,  0.0072, -0.0018, -0.0144,
         0.0139,  0.0137, -0.0066, -0.0090,  0.0050,  0.0160, -0.0044,  0.0024,
        -0.0010,  0.0025,  0.0039,  0.0011, -0.0085, -0.0055,  0.0101,  0.0090,
        -0.0054,  0.0112,  0.0011,  0.0071,  0.0143, -0.0016,  0.0084, -0.0090,
        

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [7]:


for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.307
[1,  4000] loss: 2.037
[1,  6000] loss: 1.898
[1,  8000] loss: 1.796
[1, 10000] loss: 1.740
[1, 12000] loss: 1.699
[2,  2000] loss: 1.647
[2,  4000] loss: 1.621
[2,  6000] loss: 1.583
[2,  8000] loss: 1.565
[2, 10000] loss: 1.562
[2, 12000] loss: 1.513
Finished Training


In [8]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 45 %


In [9]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 50.1 %
Accuracy for class car   is: 52.0 %
Accuracy for class bird  is: 46.3 %
Accuracy for class cat   is: 25.4 %
Accuracy for class deer  is: 28.6 %
Accuracy for class dog   is: 42.2 %
Accuracy for class frog  is: 58.9 %
Accuracy for class horse is: 40.6 %
Accuracy for class ship  is: 46.5 %
Accuracy for class truck is: 63.9 %


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [26]:
#--------------------------------------------------
#       Model Training Function
#--------------------------------------------------
import torch.optim as optim
import time
  
def trainModel(net, trainloader, train_option, testloader=None):  
  loss_func = nn.CrossEntropyLoss()
  lr = train_option['lr']
  epoch = train_option['epoch']
  device = train_option['device'] if 'device' in train_option.keys() else 'cpu'
  log_iter = train_option['log_iter'] if 'log_iter' in train_option.keys() else 1000
  eval_epoch = 1
  
  if 'optimizer' in train_option.keys():
    optimizer = train_option['optimizer']
  else:
    optimizer = optim.Adam(net.parameters(), lr=lr)

  start_time = time.time()
  if device == 'gpu':
    net = net.cuda()
    
  iters = 0
  running_loss = 0.0
  for ep in range(epoch):
    net.train()        
    for iter, (x, y) in enumerate(trainloader):
      iters += 1
      batch_x = Variable(x).float()
      batch_y = Variable(y).long()
      if device == 'gpu':
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()

      outputs = net(batch_x)
      loss = loss_func(outputs, batch_y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      
      time_lapse = time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))
      if iter % log_iter == 0:
        print('Epoch:{:2d} | Iter:{:5d} | Time: {} | Train Loss: {:.4f} | Average Loss: {:.4f} '.format(ep+1, iter, time_lapse, loss.item(), running_loss/iters))
   
    if testloader is not None and ep % eval_epoch == 0:
      evalModel(net, testloader)
  
  return net

In [14]:
#--------------------------------------------------
#       Model Evaluating Function
#--------------------------------------------------
import time
  
def evalModel(net, testloader): 
  acc = 0.0
  count = 0
  start_time = time.time()
  device = 'gpu' if next(net.parameters()).is_cuda else 'cpu'
  net.eval()
  
  for iter, (x, y) in enumerate(testloader):
        count += x.shape[0]
        batch_x = Variable(x).float()
        batch_y = Variable(y).long()
        if device == 'gpu':
          batch_x = batch_x.cuda()
          batch_y = batch_y.cuda()
        outputs = net(batch_x)
        acc += torch.sum(outputs.max(1)[1]==batch_y)
        
  time_lapse = time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))        
  print('Accuracy: {:5f} | Time: {}'.format(acc/count,time_lapse))
  

In [15]:
import torchvision.models as models
resnet =  models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [22]:
resnet.fc = nn.Linear(512,10)

In [28]:
net = resnet
train_option = {}
train_option['lr'] = 0.002
train_option['epoch'] = 2
train_option['device'] = 'gpu'
train_option['optimizer'] = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
model = trainModel(net, trainloader, train_option, testloader)

Epoch: 1 | Iter:    0 | Time: 00:00:00 | Train Loss: 2.6897 | Average Loss: 2.6897 
Epoch: 1 | Iter: 1000 | Time: 00:00:13 | Train Loss: 2.2425 | Average Loss: 1.9986 
Epoch: 1 | Iter: 2000 | Time: 00:00:26 | Train Loss: 1.8283 | Average Loss: 1.9849 
Epoch: 1 | Iter: 3000 | Time: 00:00:40 | Train Loss: 2.1314 | Average Loss: 1.9832 
Epoch: 1 | Iter: 4000 | Time: 00:00:53 | Train Loss: 1.9232 | Average Loss: 1.9730 
Epoch: 1 | Iter: 5000 | Time: 00:01:07 | Train Loss: 2.5321 | Average Loss: 1.9653 
Epoch: 1 | Iter: 6000 | Time: 00:01:20 | Train Loss: 1.6662 | Average Loss: 1.9584 
Epoch: 1 | Iter: 7000 | Time: 00:01:34 | Train Loss: 2.6095 | Average Loss: 1.9515 
Epoch: 1 | Iter: 8000 | Time: 00:01:47 | Train Loss: 2.2866 | Average Loss: 1.9408 
Epoch: 1 | Iter: 9000 | Time: 00:02:00 | Train Loss: 1.6482 | Average Loss: 1.9354 
Epoch: 1 | Iter:10000 | Time: 00:02:14 | Train Loss: 2.5226 | Average Loss: 1.9400 
Epoch: 1 | Iter:11000 | Time: 00:02:27 | Train Loss: 1.2840 | Average Loss: 

In [37]:
correct = 0
total = 0
acc = 0.0
count = 0
start_time = time.time()
device = 'gpu' if next(model.parameters()).is_cuda else 'cpu'
model.eval()
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
  for iter, (x, y) in enumerate(testloader):
        count += x.shape[0]
        batch_x = Variable(x).float()
        batch_y = Variable(y).long()
        if device == 'gpu':
          batch_x = batch_x.cuda()
          batch_y = batch_y.cuda()
        outputs = model(batch_x)
        # calculate outputs by running images through the network
        #outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 36 %


In [38]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

correct = 0
total = 0
acc = 0.0
count = 0
start_time = time.time()
device = 'gpu' if next(model.parameters()).is_cuda else 'cpu'
model.eval()

# again no gradients needed
with torch.no_grad():
   for iter, (x, y) in enumerate(testloader):
        count += x.shape[0]
        batch_x = Variable(x).float()
        batch_y = Variable(y).long()
        if device == 'gpu':
          batch_x = batch_x.cuda()
          batch_y = batch_y.cuda()
        outputs = model(batch_x)
        # calculate outputs by running images through the network
        #outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
    
        # collect the correct predictions for each class
        for label, prediction in zip(batch_y, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 0.0 %
Accuracy for class car   is: 26.0 %
Accuracy for class bird  is: 0.0 %
Accuracy for class cat   is: 25.2 %
Accuracy for class deer  is: 0.0 %
Accuracy for class dog   is: 0.0 %
Accuracy for class frog  is: 22.8 %
Accuracy for class horse is: 25.6 %
Accuracy for class ship  is: 0.0 %
Accuracy for class truck is: 0.0 %


In [39]:
######################### Feature Extraction ####################3
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

model_conv.fc = nn.Linear(512,10)

In [40]:
net = model_conv
train_option = {}
train_option['lr'] = 0.002
train_option['epoch'] = 2
train_option['device'] = 'gpu'
train_option['optimizer'] = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
model = trainModel(net, trainloader, train_option, testloader)

Epoch: 1 | Iter:    0 | Time: 00:00:00 | Train Loss: 2.9948 | Average Loss: 2.9948 
Epoch: 1 | Iter: 1000 | Time: 00:00:06 | Train Loss: 22.2760 | Average Loss: 16.2597 
Epoch: 1 | Iter: 2000 | Time: 00:00:12 | Train Loss: 23.0706 | Average Loss: 17.7665 
Epoch: 1 | Iter: 3000 | Time: 00:00:18 | Train Loss: 11.3691 | Average Loss: 18.3549 
Epoch: 1 | Iter: 4000 | Time: 00:00:24 | Train Loss: 6.9758 | Average Loss: 18.7525 
Epoch: 1 | Iter: 5000 | Time: 00:00:30 | Train Loss: 9.4908 | Average Loss: 19.0309 
Epoch: 1 | Iter: 6000 | Time: 00:00:36 | Train Loss: 22.4392 | Average Loss: 19.1761 
Epoch: 1 | Iter: 7000 | Time: 00:00:42 | Train Loss: 6.3635 | Average Loss: 19.3364 
Epoch: 1 | Iter: 8000 | Time: 00:00:48 | Train Loss: 29.1681 | Average Loss: 19.5572 
Epoch: 1 | Iter: 9000 | Time: 00:00:54 | Train Loss: 18.4961 | Average Loss: 19.6452 
Epoch: 1 | Iter:10000 | Time: 00:01:00 | Train Loss: 10.8940 | Average Loss: 19.7351 
Epoch: 1 | Iter:11000 | Time: 00:01:06 | Train Loss: 31.198

In [41]:
correct = 0
total = 0
acc = 0.0
count = 0
start_time = time.time()
device = 'gpu' if next(model.parameters()).is_cuda else 'cpu'
model.eval()
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
  for iter, (x, y) in enumerate(testloader):
        count += x.shape[0]
        batch_x = Variable(x).float()
        batch_y = Variable(y).long()
        if device == 'gpu':
          batch_x = batch_x.cuda()
          batch_y = batch_y.cuda()
        outputs = model(batch_x)
        # calculate outputs by running images through the network
        #outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 20 %


In [42]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

correct = 0
total = 0
acc = 0.0
count = 0
start_time = time.time()
device = 'gpu' if next(model.parameters()).is_cuda else 'cpu'
model.eval()

# again no gradients needed
with torch.no_grad():
   for iter, (x, y) in enumerate(testloader):
        count += x.shape[0]
        batch_x = Variable(x).float()
        batch_y = Variable(y).long()
        if device == 'gpu':
          batch_x = batch_x.cuda()
          batch_y = batch_y.cuda()
        outputs = model(batch_x)
        # calculate outputs by running images through the network
        #outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
    
        # collect the correct predictions for each class
        for label, prediction in zip(batch_y, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 0.0 %
Accuracy for class car   is: 26.0 %
Accuracy for class bird  is: 0.0 %
Accuracy for class cat   is: 25.2 %
Accuracy for class deer  is: 0.0 %
Accuracy for class dog   is: 0.0 %
Accuracy for class frog  is: 22.8 %
Accuracy for class horse is: 25.6 %
Accuracy for class ship  is: 0.0 %
Accuracy for class truck is: 0.0 %
